In [1]:
import pandas as pd
import numpy as np

In [2]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

##### pivot_table(data,values=None,index=None,columns=None,aggfunc='mean',margins=False,margins_name='All')

- data : 분석할 데이터 프레임. 메서드 형식일때는 필요하지 않음 ex)df1.pivot_table()  
- values : 분석할 데이터 프레임에서 분석할 열  
- index : 행 인덱스로 들어갈 키열 또는 키열의 리스트  
- columns : 열 인덱스로 들어갈 키열 또는 키열의 리스트  
- fill_value : NaN이 표출될 때 대체값 지정  
- margins : 모든 데이터를 분석한 결과를 행으로 표출할 지 여부  
- margins_name : margins가 표출될 때 그 열(행)의 이름

##### 피봇테이블을 작성할 때 반드시 설정해야 되는 인수
- data : 사용 데이터 프레임
- index : 행 인덱스로 사용할 필드(기준 필드로 작용됨)
- 인덱스 명을 제외한 나머지 값(data)은 수치 data 만 사용함
- 기본 함수가 평균(mean)함수 이기 때문에 각 데이터의 평균값이 반환

In [3]:
data = {
    "도시": ["서울", "서울", "서울", "부산", "부산", "부산", "인천", "인천"],
    "연도": ["2015", "2010", "2005", "2015", "2010", "2005", "2015", "2010"],
    "인구": [9904312, 9631482, 9762546, 3448737, 3393191, 3512547, 2890451, 263203],
    "지역": ["수도권", "수도권", "수도권", "경상권", "경상권", "경상권", "수도권", "수도권"]
}

columns = ["도시", "연도", "인구", "지역"]
df1 = pd.DataFrame(data, columns=columns)
df1

,도시,연도,인구,지역
0,서울,2015,9904312,수도권
1,서울,2010,9631482,수도권
2,서울,2005,9762546,수도권
3,부산,2015,3448737,경상권
4,부산,2010,3393191,경상권
5,부산,2005,3512547,경상권
6,인천,2015,2890451,수도권
7,인천,2010,263203,수도권


In [5]:
# 각 도시에 대한 연도별 인구 '평균'

df1.pivot_table(index = "도시",columns='연도',values='인구')

연도,2005,2010,2015
도시,,,
부산,3512547.0,3393191.0,3448737.0
서울,9762546.0,9631482.0,9904312.0
인천,NaN,263203.0,2890451.0


In [6]:
#각 지역별 도시에 대한 연도별 인구 평균
df1.pivot(index=['지역','도시'], columns='연도', values ='인구')

연도           2005       2010       2015
지역  도시                                 
경상권 부산  3512547.0  3393191.0  3448737.0
수도권 서울  9762546.0  9631482.0  9904312.0
    인천        NaN   263203.0  2890451.0

### seaborn 의 타이타닉 데이터로 피벗테이블 만들기

In [ ]:
pip install seaborn

In [6]:
import pandas as pd
import seaborn as sns

df = sns.load_dataset('titanic')[['age','sex','class','fare','survived']]
df.head()

,age,sex,class,fare,survived
0,22.0,male,Third,7.2500,0
1,38.0,female,First,71.2833,1
2,26.0,female,Third,7.9250,1
3,35.0,female,First,53.1000,1
4,35.0,male,Third,8.0500,0


In [8]:
#선실 등급별로 숙박객의 성별 평균 나이
pdf1 = pd.pivot_table(df, index='class', columns = 'sex', values = 'age', aggfunc='mean')
pdf1

sex,female,male
class,,
First,34.611765,41.281386
Second,28.722973,30.740707
Third,21.750000,26.507589


##### 피벗 테이블로 여러 조건 추가하여 다양한 계산 하기
- 다수의 조건 넣을 시 [] 사용

In [10]:
#선실 등급별 숙박객의 생존자 수와 생존율
pdf1 = pd.pivot_table(df, index='class', columns = 'sex', values = 'survived', aggfunc=['mean','sum'])
pdf1


mean              sum     
sex       female      male female male
class                                 
First   0.968085  0.368852     91   45
Second  0.921053  0.157407     70   17
Third   0.500000  0.135447     72   47

In [12]:
#선실 등급에 따른 성별(index)에 대해 생존여부별(survived)로 나이와 티켓값(values)의 평균과 최대값(aggfunc)을 산출

pdf3 = pd.pivot_table(df, index=['class','sex'], columns=['survived'],values=['age','fare'],aggfunc=['mean','max'])
pdf3

mean                                      max        \
                     age                   fare               age         
survived               0          1           0           1     0     1   
class  sex                                                                
First  female  25.666667  34.939024  110.604167  105.978159  50.0  63.0   
       male    44.581967  36.248000   62.894910   74.637320  71.0  80.0   
Second female  36.000000  28.080882   18.250000   22.288989  57.0  55.0   
       male    33.369048  16.022000   19.488965   21.095100  70.0  62.0   
Third  female  23.818182  19.329787   19.773093   12.464526  48.0  63.0   
       male    27.255814  22.274211   12.204469   15.579696  74.0  45.0   

                                 
                 fare            
survived            0         1  
class  sex                       
First  female  151.55  512.3292  
       male    263.00  512.3292  
Second female   26.00   65.0000  
       male     73.50   39.0000  
Third  female   69.55   31.3875  
       male     69.55   56.4958

### 그룹분석

- 만약 키가 지정하는 조건에 맞는 데이터가 하나 이상이라서 데이터 그룹을 이루는 경우에는 그룹의 특성을 보여주는 그룹분석(group analysis)을 해야 함

    - 그룹분석은 피봇테이블과 달리 키에 의해서 결정되는 데이터가 여러개가 있을 경우 미리 지정한 연산을 통해 그 그룹 데이터의 대표값을 계산 하는 것


- 판다스에서는 groupby 메서드를 사용하여 아래 내용 처럼 그룹분석을 진행

    - 분석하고자 하는 시리즈나 데이터프레임에 groupby 메서드를 호출하여 그룹화 수행

    - 그룹 객체에 대해 그룹연산을 수행

#### groupby 메서드¶
- groupby 메서드는 데이터를 그룹 별로 분류하는 역할을 함 

- groupby 메서드의 인수

    - 열 또는 열의 리스트

    - 행 인덱스

- 연산 결과로 그룹 데이터를 나타내는 GroupBy 클래스 객체를 반환
    - 이 객체에는 그룹별로 연산을 할 수 있는 그룹연산 메서드가 있음

#### GroupBy 클래스 객체의 그룹연산 메서드

- size, count: 그룹 데이터의 갯수

- mean, median, min, max: 그룹 데이터의 평균, 중앙값, 최소, 최대

- sum, prod, std, var, quantile : 그룹 데이터의 합계, 곱, 표준편차, 분산, 사분위수

- first, last: 그룹 데이터 중 가장 첫번째 데이터와 가장 나중 데이터


#### 이 외에도 많이 사용되는 그룹 연산

- agg, aggregate

    - 만약 원하는 그룹연산이 없는 경우 함수를 만들고 이 함수를 agg에 전달한다.

    - 또는 여러가지 그룹연산을 동시에 하고 싶은 경우 함수 이름 문자열의 리스트를 전달한다.

- describe

    - 하나의 그룹 대표값이 아니라 여러개의 값을 데이터프레임으로 구한다.

- apply

    - describe 처럼 하나의 대표값이 아닌 데이터프레임을 출력하지만 원하는 그룹연산이 없는 경우에 사용한다.

- transform

    - 그룹에 대한 대표값을 만드는 것이 아니라 그룹별 계산을 통해 데이터 자체를 변형한다.

In [15]:
import numpy as np


np.random.seed(0)
df2 = pd.DataFrame({
    'key1': ['A', 'A', 'B', 'B', 'A'],
    'key2': ['one', 'two', 'one', 'two', 'one'],
    'data1': [1, 2, 3, 4, 5],
    'data2': [10, 20, 30, 40, 50]})
df2

,key1,key2,data1,data2
0,A,one,1,10
1,A,two,2,20
2,B,one,3,30
3,B,two,4,40
4,A,one,5,50


In [17]:
df2_group=df2.groupby(df2.key1)
df2_group 
#<pandas.core.groupby.generic.DataFrameGroupBy object at 0x0000029DE78A94E0> = 데이터프레임 결과입니다 라는 출력값

In [20]:
#groups 속성 확인 : 그룹핑 된 요약 보기
df2_group.groups

{'A': [0, 1, 4], 'B': [2, 3]}

In [23]:
#딕셔너리 반환 {그룹명 : 그룹에 포함된 행인덱스}
df2_group.groups.keys()

dict_keys(['A', 'B'])

In [24]:
df2_group.groups['A']   # key 값(그룹명) 적용함으로써 인덱스 추출

#Int64Index([0, 1, 4], dtype='int64') 

Int64Index([0, 1, 4], dtype='int64')

In [26]:
pd.DataFrame(df2_group)

,0,1
0,A,key1 key2 data1 data2 0 A one 1 ...
1,B,key1 key2 data1 data2 2 B one 3 ...


In [27]:
#A 그룹만 추출
pd.DataFrame(df2_group).loc[0].values 

#위 데이터프레임에서 0 행 추출 


array(['A',   key1 key2  data1  data2
            0    A  one      1     10
            1    A  two      2     20
            4    A  one      5     50], dtype=object)

In [28]:
#B 그룹만 추출
pd.DataFrame(df2_group).loc[1].values 

# 위 데이터프레임에서 1 행 추출

array(['B',   key1 key2  data1  data2
            2    B  one      3     30
            3    B  two      4     40], dtype=object)

In [30]:
#데이터 프레임 형태로
pd.DataFrame(df2_group).loc[1].values[1]

,key1,key2,data1,data2
2,B,one,3,30
3,B,two,4,40


In [31]:
#groupby 메서드에 연산 메서드 적용시켜 요약 
df2_group.sum()

,data1,data2
key1,,
A,8,80
B,7,70


In [33]:
#컬럼 추출 - data1 선택 후 계산 
df2_group['data1'].sum()

key1
A    8
B    7
Name: data1, dtype: int64

In [34]:
#컬럼 추출 - 전체 데이터 계산 후 data1 선택
df2_group.sum()['data1']

key1
A    8
B    7
Name: data1, dtype: int64

In [36]:
#데이터 프레임 출력
df2_group[['data1']].sum()

,data1
key1,
A,8
B,7
